In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from PIL import Image
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
import os
from torch.utils import data
import random
import copy
import itertools
import io

# Tensorboard for logging
from torch.utils.tensorboard import SummaryWriter



In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda') 
else:
    device = torch.device('cpu')
    
torch.cuda.set_device(0)
print(device)

cuda


In [3]:
# root directory
root_dir = "/home/andreasabo/Documents/HNProject/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/andreasabo/Documents/HNProject/all_label_img/"

# read target df
csv_path = os.path.join(root_dir, "all_splits_100000.csv")
data_df = pd.read_csv(csv_path, usecols=['subj_id', 'image_ids', 'view_label', 'view_train'])
data_df.head

<bound method NDFrame.head of        image_ids     view_label  subj_id  view_train
0       1323_2_1        Missing     1323         NaN
1       1323_2_2        Missing     1323         NaN
2       1323_2_3        Missing     1323         NaN
3       1323_2_4        Missing     1323         NaN
4       1323_2_5        Missing     1323         NaN
...          ...            ...      ...         ...
72454  1066_7_50  Saggital_Left     1066         0.0
72455  1066_7_51  Saggital_Left     1066         0.0
72456  1066_7_52  Saggital_Left     1066         0.0
72457  1066_7_53  Saggital_Left     1066         0.0
72458  1066_7_54          Other     1066         0.0

[72459 rows x 4 columns]>

### **Hyperparameters**

In [4]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset: right_sag, right_trav, left_sag, left_trav, bladder, other
num_classes = 6

# Batch size for training (change depending on how much memory you have)
batch_size = 48

# Number of epochs to train for
num_epochs = 100

# Flag for feature extracting. When False, we finetune the whole model; when True we only update the reshaped layer params
feature_extract = False

### **Reading Data Indicies and Labels**

In [5]:
label_mapping = {'Other':0, 'Saggital_Right':1, 'Transverse_Right':2, 
                 'Saggital_Left':3, 'Transverse_Left':4, 'Bladder':5}

data_df['view_label'] = data_df['view_label'].map(label_mapping)

train_df = data_df[data_df.view_train == 1]
test_df = data_df[data_df.view_train == 0]

labels = {}
train_and_valid_subj_ids = []
train_and_valid_image_ids = []
test_ids = []

for ind, row in train_df.iterrows():
    train_and_valid_subj_ids.append(row['subj_id'])
    train_and_valid_image_ids.append(row['image_ids'])
    labels[row['image_ids']] = row['view_label']

for ind, row in test_df.iterrows():
    test_ids.append(row['image_ids'])
    labels[row['image_ids']] = row['view_label']

s = set()
t_v_ids = pd.DataFrame(list(zip(train_and_valid_subj_ids, train_and_valid_image_ids)), columns=['subj_ids', 'image_ids'])
id_groups = [t_v_ids for _, t_v_ids in t_v_ids.groupby('subj_ids')]
random.shuffle(id_groups)
id_groups = pd.concat(id_groups).reset_index(drop=True)
train_val_split = int(0.8*len(set(id_groups['subj_ids'].values)))
train_val_set = [i for i in id_groups['subj_ids'].values if not (i in s or s.add(i))]
cutoff = train_val_set[train_val_split]
train_portion = (id_groups['subj_ids'].values == cutoff).argmax()

train_ids = id_groups[:train_portion]['image_ids'].tolist()
valid_ids = id_groups[train_portion:]['image_ids'].tolist()

partition = {'train':train_ids, 'valid':valid_ids}

In [6]:
def plot_confusion_matrix_local(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [7]:
# https://www.tensorflow.org/tensorboard/image_summaries
# def plot_to_image(figure):
#   """Converts the matplotlib plot specified by 'figure' to a PNG image and
#   returns it. The supplied figure is closed and inaccessible after this call."""
#   # Save the plot to a PNG in memory.
#   buf = io.BytesIO()
#   plt.savefig(buf, format='png')
#   # Closing the figure prevents it from being displayed directly inside
#   # the notebook.
#   plt.close(figure)
#   buf.seek(0)
#   # Convert PNG buffer to TF image
#   image = tf.image.decode_png(buf.getvalue(), channels=4)
#   # Add the batch dimension
#   image = tf.expand_dims(image, 0)
#   return image

### **Model**

In [8]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False, writer = None):
    since = time.time()
    classnames = ['Other', 'Saggital_Right', 'Transverse_Right', 'Saggital_Left','Transverse_Left', 'Bladder']
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 54)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            running_preds = []
            running_labels = []

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                labels = labels.type(torch.long)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        labels = torch.argmax(labels, 1)
                        running_preds += torch.argmax(outputs, 1).tolist()
                        running_labels += labels.tolist()
                        loss = criterion(outputs, labels)

                    preds = torch.argmax(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} loss:\t{:.4f} | {} acc:\t{:.4f}\n'.format(phase, epoch_loss, phase, epoch_acc))
            # Log to tensorboard for visualization
            if writer is not None and phase == 'train':
                writer.add_scalar('Loss/train', epoch_loss, epoch)
                writer.add_scalar('Accuracy/train', epoch_acc, epoch)
#                 cm = confusion_matrix(running_labels, running_preds)
#                 figure = plot_confusion_matrix_local(cm, classnames)
#                 writer.add_image('confusion_matrix/train', figure, epoch)
                
                
                
            if writer is not None and phase == 'val':
                writer.add_scalar('Loss/val', epoch_loss, epoch)
                writer.add_scalar('Accuracy/val', epoch_acc, epoch)
                
#                 cm = confusion_matrix(running_labels, running_preds)
#                 figure = plot_confusion_matrix_local(cm, classnames)
#                 writer.add_image('confusion_matrix/val', figure, epoch)
            # deep copy the model
            if phase == 'train':
                print(classification_report(running_labels, running_preds))
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(classification_report(running_labels, running_preds))
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [10]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)

class ViewNet(nn.Module):

    
    def __init__(self, num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout):
        super(ViewNet, self).__init__()
        self.conv1_filters = conv1_filters
        self.conv2_filters = conv2_filters
        self.conv3_filters = conv3_filters
        self.linear1_size = linear1_size
        self.drop_percent = dropout
        
        self.conv1 = nn.Conv2d(1, self.conv1_filters, 4, padding=2)
        self.conv2 = nn.Conv2d(self.conv1_filters, self.conv2_filters, 4, padding=2)
        self.conv3 = nn.Conv2d(self.conv2_filters, self.conv3_filters, 4, padding=2)
        self.pool = nn.MaxPool2d(4, 4)
        self.dropout = nn.Dropout(self.drop_percent)
        self.linear1 = nn.Linear(self.conv3_filters*4*4, self.linear1_size)
        self.dropout = nn.Dropout(self.drop_percent)
        self.linear2 = nn.Linear(self.linear1_size, num_classes)
    
    def forward(self, x):
        x = self.pool(self.dropout(F.relu(self.conv1(x))))
        x = self.pool(self.dropout(F.relu(self.conv2(x))))
        x = self.pool(self.dropout(F.relu(self.conv3(x))))
        x = x.view(-1, self.conv3_filters*4*4) ## reshaping 
        x = self.dropout(F.relu((self.linear1(x))))
        x = self.linear2(x)
        return x



### **Data Loading**

In [11]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        img_path = data_dir + ID + '.jpg'
        image = Image.open(img_path).convert('L')
        image = ToTensor()(image)
        y = torch.FloatTensor([0]*6)
        y[int(self.labels[ID])] = 1

        return image, y

In [12]:
# Data augmentation and normalization for training

# Parameters
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = Dataset(partition['train'], labels)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['valid'], labels)
validation_generator = data.DataLoader(validation_set, **params)

dataloaders_dict = {'train':training_generator, 'val':validation_generator}

### **Running the Model**

In [13]:
def SetupAndRunTest(model_ft, writer, lr, wd, amsgrad):
    
    input_size = 256

    # Print the model we just instantiated
#     print(model_ft)

    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
#     print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.Adam(params_to_update, lr=lr, weight_decay=wd, amsgrad=amsgrad)

    # Setup the loss fxn
    criterion = nn.CrossEntropyLoss()
    
    model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs, is_inception=(model_name=="inception"), writer = writer)

In [14]:
# Train and evaluate
# model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs, is_inception=(model_name=="inception"), writer = writer)

In [15]:
feature_extract

False

In [ ]:
writer_prefix = "mar11_ablation"
conv1_filters_size = [8, 16, 32]
conv2_filters_size = [16, 32, 8]
conv3_filters_size = [32, 16, 8]
linear1_sizes = [256, 512, 1024]
dropouts = [0.3, 0.2]

# lrs = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]
# weight_decays = [1e-5, 5e-5, 1e-4, 5e-4, 1e-6, 5e-6]
lrs = [5e-4, 1e-3, 1e-4]
weight_decays = [5e-3]

amsgrads=[False]
for conv1_filters in conv1_filters_size:
    for conv2_filters in conv2_filters_size:
        for conv3_filters in conv3_filters_size:
            for linear1_size in linear1_sizes:
                for dropout in dropouts:
                    for lr in lrs:
                        for wd in weight_decays:
                            for amsgrad in amsgrads:
                                config_string = f"{conv1_filters}_{conv2_filters}_{conv3_filters}_{linear1_size}_{dropout}_{lr}_{wd}_{amsgrad}"

                                model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
                                writer = SummaryWriter(writer_prefix + "/" + config_string)
                                SetupAndRunTest(model_ft, writer, lr, wd, amsgrad)



	 conv1.weight
	 conv1.bias
	 conv2.weight
	 conv2.bias
	 conv3.weight
	 conv3.bias
	 linear1.weight
	 linear1.bias
	 linear2.weight
	 linear2.bias
Epoch 1/100
------------------------------------------------------
train loss:	1.4894 | train acc:	0.4313

              precision    recall  f1-score   support

           0       0.44      0.90      0.59      4725
           1       0.35      0.12      0.18      1805
           2       0.55      0.01      0.02      1275
           3       0.33      0.18      0.23      1752
           4       0.35      0.02      0.04      1317
           5       0.56      0.32      0.41      1015

    accuracy                           0.43     11889
   macro avg       0.43      0.26      0.24     11889
weighted avg       0.42      0.43      0.34     11889

val loss:	1.5235 | val acc:	0.4679

              precision    recall  f1-score   support

           0       0.40      0.88      0.55       736
           1       0.64      0.13      0.22       334
   

train loss:	1.0572 | train acc:	0.5966

              precision    recall  f1-score   support

           0       0.59      0.77      0.67      4725
           1       0.63      0.49      0.55      1805
           2       0.59      0.35      0.44      1275
           3       0.52      0.49      0.50      1752
           4       0.56      0.40      0.47      1317
           5       0.75      0.75      0.75      1015

    accuracy                           0.60     11889
   macro avg       0.61      0.54      0.56     11889
weighted avg       0.60      0.60      0.59     11889

val loss:	1.3359 | val acc:	0.5172

Epoch 10/100
------------------------------------------------------
train loss:	1.0325 | train acc:	0.6015

              precision    recall  f1-score   support

           0       0.59      0.76      0.67      4725
           1       0.62      0.48      0.54      1805
           2       0.61      0.37      0.46      1275
           3       0.53      0.49      0.51      1752
  

val loss:	1.2840 | val acc:	0.5585

              precision    recall  f1-score   support

           0       0.48      0.78      0.60       736
           1       0.60      0.48      0.53       334
           2       0.53      0.40      0.46       220
           3       0.63      0.37      0.47       423
           4       0.58      0.41      0.48       321
           5       0.90      0.67      0.77       238

    accuracy                           0.56      2272
   macro avg       0.62      0.52      0.55      2272
weighted avg       0.59      0.56      0.55      2272

Epoch 20/100
------------------------------------------------------
train loss:	0.8456 | train acc:	0.6793

              precision    recall  f1-score   support

           0       0.66      0.79      0.72      4725
           1       0.72      0.58      0.65      1805
           2       0.69      0.54      0.60      1275
           3       0.63      0.61      0.62      1752
           4       0.64      0.53      0.5

val loss:	1.2053 | val acc:	0.5524

Epoch 31/100
------------------------------------------------------
train loss:	0.6612 | train acc:	0.7514

              precision    recall  f1-score   support

           0       0.74      0.82      0.78      4725
           1       0.77      0.68      0.72      1805
           2       0.76      0.67      0.71      1275
           3       0.72      0.70      0.71      1752
           4       0.71      0.66      0.69      1317
           5       0.85      0.86      0.86      1015

    accuracy                           0.75     11889
   macro avg       0.76      0.73      0.74     11889
weighted avg       0.75      0.75      0.75     11889

val loss:	1.2112 | val acc:	0.5458

Epoch 32/100
------------------------------------------------------
train loss:	0.6472 | train acc:	0.7546

              precision    recall  f1-score   support

           0       0.75      0.82      0.78      4725
           1       0.77      0.70      0.73      1805
      

val loss:	1.2104 | val acc:	0.5409

Epoch 43/100
------------------------------------------------------
train loss:	0.5292 | train acc:	0.8012

              precision    recall  f1-score   support

           0       0.80      0.85      0.83      4725
           1       0.81      0.76      0.79      1805
           2       0.79      0.72      0.75      1275
           3       0.78      0.76      0.77      1752
           4       0.79      0.74      0.76      1317
           5       0.87      0.87      0.87      1015

    accuracy                           0.80     11889
   macro avg       0.81      0.79      0.79     11889
weighted avg       0.80      0.80      0.80     11889

val loss:	1.1976 | val acc:	0.5462

Epoch 44/100
------------------------------------------------------
train loss:	0.5317 | train acc:	0.7996

              precision    recall  f1-score   support

           0       0.80      0.85      0.82      4725
           1       0.80      0.75      0.78      1805
      

...

val loss:	1.4462 | val acc:	0.4762

Epoch 10/100
------------------------------------------------------
train loss:	1.2255 | train acc:	0.5286

              precision    recall  f1-score   support

           0       0.53      0.79      0.64      4725
           1       0.52      0.42      0.46      1805
           2       0.55      0.15      0.23      1275
           3       0.45      0.37      0.41      1752
           4       0.46      0.21      0.29      1317
           5       0.66      0.66      0.66      1015

    accuracy                           0.53     11889
   macro avg       0.53      0.43      0.45     11889
weighted avg       0.52      0.53      0.50     11889

val loss:	1.4487 | val acc:	0.5189

              precision    recall  f1-score   support

           0       0.46      0.85      0.59       736
           1       0.50      0.37      0.43       334
           2       0.72      0.17      0.28       220
           3       0.52      0.36      0.43       423
      

train loss:	1.1253 | train acc:	0.5701

              precision    recall  f1-score   support

           0       0.57      0.78      0.65      4725
           1       0.57      0.46      0.51      1805
           2       0.60      0.28      0.38      1275
           3       0.51      0.43      0.47      1752
           4       0.50      0.32      0.39      1317
           5       0.71      0.73      0.72      1015

    accuracy                           0.57     11889
   macro avg       0.58      0.50      0.52     11889
weighted avg       0.57      0.57      0.55     11889

val loss:	1.3728 | val acc:	0.5246

Epoch 20/100
------------------------------------------------------
train loss:	1.1145 | train acc:	0.5738

              precision    recall  f1-score   support

           0       0.57      0.79      0.66      4725
           1       0.57      0.45      0.50      1805
           2       0.59      0.28      0.38      1275
           3       0.51      0.43      0.47      1752
  

val loss:	1.3551 | val acc:	0.5321

Epoch 30/100
------------------------------------------------------
train loss:	1.0410 | train acc:	0.6024

              precision    recall  f1-score   support

           0       0.60      0.78      0.68      4725
           1       0.62      0.49      0.55      1805
           2       0.62      0.34      0.44      1275
           3       0.54      0.48      0.51      1752
           4       0.56      0.40      0.46      1317
           5       0.73      0.75      0.74      1015

    accuracy                           0.60     11889
   macro avg       0.61      0.54      0.56     11889
weighted avg       0.60      0.60      0.59     11889

val loss:	1.3635 | val acc:	0.5238

Epoch 31/100
------------------------------------------------------
train loss:	1.0352 | train acc:	0.6069

              precision    recall  f1-score   support

           0       0.60      0.78      0.68      4725
           1       0.64      0.50      0.56      1805
      

train loss:	0.9684 | train acc:	0.6383

              precision    recall  f1-score   support

           0       0.63      0.79      0.70      4725
           1       0.67      0.55      0.61      1805
           2       0.64      0.42      0.51      1275
           3       0.58      0.53      0.56      1752
           4       0.61      0.46      0.52      1317
           5       0.77      0.78      0.78      1015

    accuracy                           0.64     11889
   macro avg       0.65      0.59      0.61     11889
weighted avg       0.64      0.64      0.63     11889

val loss:	1.3165 | val acc:	0.5321

Epoch 42/100
------------------------------------------------------
train loss:	0.9669 | train acc:	0.6337

              precision    recall  f1-score   support

           0       0.63      0.79      0.70      4725
           1       0.66      0.53      0.59      1805
           2       0.64      0.42      0.51      1275
           3       0.58      0.52      0.55      1752
  

train loss:	0.9122 | train acc:	0.6536

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      4725
           1       0.68      0.56      0.61      1805
           2       0.66      0.46      0.54      1275
           3       0.60      0.55      0.58      1752
           4       0.61      0.49      0.54      1317
           5       0.79      0.79      0.79      1015

    accuracy                           0.65     11889
   macro avg       0.67      0.61      0.63     11889
weighted avg       0.65      0.65      0.65     11889

val loss:	1.2929 | val acc:	0.5383

Epoch 54/100
------------------------------------------------------
train loss:	0.8962 | train acc:	0.6599

              precision    recall  f1-score   support

           0       0.64      0.81      0.72      4725
           1       0.70      0.56      0.62      1805
           2       0.68      0.46      0.55      1275
           3       0.61      0.57      0.59      1752
  

Comparing without Transfer Learning

In [ ]:
# # Initialize the non-pretrained version of the model used for this run
# scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
# scratch_model = scratch_model.to(device)
# scratch_optimizer = optim.Adam(scratch_model.parameters())
# scratch_criterion = nn.CrossEntropyLoss()
# _,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# # Plot the training curves of validation accuracy vs. number
# #  of training epochs for the transfer learning method and
# #  the model trained from scratch
# ohist = []
# shist = []

# ohist = [h.cpu().numpy() for h in hist]
# shist = [h.cpu().numpy() for h in scratch_hist]

# plt.title("Validation Accuracy vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Validation Accuracy")
# plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),shist,label="Scratch")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()

THE END! :D

In [ ]:
# Use dataloader to load the data

# Read data from file

# tansform the data into the desired shape


# split on the train and test


# convert into tensor



# Data Loading
batch_size = 100

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
# Define a small CNN that processes 2-channel images
# and output the view label of the image, the output should be one of five labels
# kidney_right_sag, kidney_right_trav, kidney_left_sag, kidney_left_trav, bladder

class Net(nn.Module):
    # Initialize our layers, i.e. the set of trainable parameters
    def __init__(self):
        super(Net, self).__init__()
        # A convolutional layer
        # The number of input channels is 2 greyscale
        self.conv1 = nn.Conv2d(2, 6, 5)
        # A max pooling layer (will be reused for each conv layer)
        self.pool = nn.MaxPool2d(2, 2)
        # Another convolutional layer
        self.conv2 = nn.Conv2d(4, 16, 5)
        # Three sets of fully connected (linear) layers 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        # For each conv layer: conv -> relu -> pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Reshape from higher dimensional tensor to a vector for the FC layers
        x = x.view(-1, 16 * 5 * 5)
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
model = model.to(device)
model

In [ ]:
# Look at the network structure
print(model.state_dict().keys())
print("Conv1:",model.conv1.weight.size())
print("Conv2:",model.conv2.weight.size())
print("fc1:",model.fc1.weight.size())
print("fc2:",model.fc2.weight.size())
print("fc3:",model.fc3.weight.size())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):
    running_loss = 0.0
    num_total = 0.0
    num_correct = 0.0

    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)              # Forward
        loss = criterion(outputs, labels)  # Compute loss
        optimizer.zero_grad()              # Zero the parameter gradients
        loss.backward()                    # Backward
        optimizer.step()                   # Gradient step

        num_correct += (torch.argmax(outputs, dim=1) == labels).sum().float()
        num_total += labels.size(0)
        
        # Print statistics
        running_loss += loss.item()
        if (i+1) % 1000 == 0:    # print every 2000 mini-batches
            print('[{}, {:5d}] loss: {:6.4f}, acc: {:6.4f}'.format(
                   epoch + 1, i + 1, running_loss / 2000, num_correct / num_total))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Overall accuracy on the test set
correct, total = 0, 0
for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    predicted = torch.argmax(outputs, dim=1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('Accuracy on test set: %d %%' % (100 * correct / total))